## Фреймворк PyTorch для разработки искусственных нейронных сетей

### Домашнее задание №3

**Dataset, Dataloader, BatchNorm, Dropout, Оптимизация**

Будем практиковаться на датасете недвижимости (sklearn.datasets.fetch_california_housing)

Ваша задача:
1. Создать Dataset для загрузки данных
2. Обернуть его в Dataloader
3. Написать архитектуру сети, которая предсказывает стоимость недвижимости. Сеть должна включать BatchNorm слои и Dropout (или НЕ включать, но нужно обосновать)
4. Сравните сходимость Adam, RMSProp и SGD, сделайте вывод по качеству работы модели

train-test разделение нужно сделать с помощью sklearn random_state=13, test_size = 0.25

**Библиотеки**

In [1]:
import math
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

**Классы и функции**

In [3]:
class HouseDataset(torch.utils.data.Dataset):
    def __init__(self, init_dataset, init_target):
        self._base_dataset = init_dataset
        self._base_target = init_target

    def __len__(self):
        return len(self._base_dataset)

    def __getitem__(self, index):
        house, target = np.array(self._base_dataset[index]), np.array(self._base_target[index])
        return house, target

### Dataset

In [4]:
california_housing = fetch_california_housing(as_frame=True)
california_housing = california_housing.frame
california_housing.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [5]:
california_housing.shape

(20640, 9)

Проверка на пустые значения

In [6]:
california_housing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   MedInc       20640 non-null  float64
 1   HouseAge     20640 non-null  float64
 2   AveRooms     20640 non-null  float64
 3   AveBedrms    20640 non-null  float64
 4   Population   20640 non-null  float64
 5   AveOccup     20640 non-null  float64
 6   Latitude     20640 non-null  float64
 7   Longitude    20640 non-null  float64
 8   MedHouseVal  20640 non-null  float64
dtypes: float64(9)
memory usage: 1.4 MB


In [7]:
california_housing.columns

Index(['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup',
       'Latitude', 'Longitude', 'MedHouseVal'],
      dtype='object')

**Разделение на трейн и тест**

In [8]:
X = california_housing

In [9]:
target_name = 'MedHouseVal'

In [10]:
y = X[target_name].values
X = X.drop(columns=target_name)
X = X.values

In [11]:
X = torch.tensor(X, dtype=torch.float)
y = torch.tensor(y, dtype=torch.float)

In [12]:
X.shape, y.shape

(torch.Size([20640, 8]), torch.Size([20640]))

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, random_state=13, test_size = 0.25)

In [14]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape 

(torch.Size([15480, 8]),
 torch.Size([5160, 8]),
 torch.Size([15480]),
 torch.Size([5160]))

**Создание Dataset для загрузки данных**

In [15]:
dataset = HouseDataset(X_train, y_train)

In [16]:
# проверка
X_train[0],y_train[0]
dataset[0]

(array([ 3.5174000e+00,  3.6000000e+01,  4.5479450e+00,  1.0943683e+00,
         1.3570000e+03,  2.0654490e+00,  3.4209999e+01, -1.1823000e+02],
       dtype=float32),
 array(2.68, dtype=float32))

In [17]:
test_dataset = HouseDataset(X_test, y_test)

**Создание Dataloader**

In [18]:
batch_size = 10
length = 8

In [19]:
train_loader = torch.utils.data.DataLoader(dataset,
                          batch_size=128,
                          shuffle=True)

In [20]:
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                          shuffle=False)

**Создание архитектуры сети**

In [21]:
class Perceptron(nn.Module):
    def __init__(self, input_dim, output_dim, activation="relu"):
        super().__init__()
        self.fc = nn.Linear(input_dim, output_dim)
        self.activation = activation
        
    def forward(self, x):
        x = self.fc(x)
        if self.activation == "relu":
            return F.relu(x)
        if self.activation == "relu":
            return F.relu(x)
        raise RuntimeError
        

class FeedForward(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        self.fc1 = Perceptron(input_dim, hidden_dim)
        self.bn = nn.BatchNorm1d(hidden_dim)
        self.dp = nn.Dropout(0.25)
        self.fc2 = Perceptron(hidden_dim, 1, "relu")
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.dp(x)
        x = self.bn(x)
        x = self.fc2(x)
        return x

*SGD*

In [22]:
net = FeedForward(length, batch_size*8)
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)
criterion = torch.nn.MSELoss()

In [23]:
import torch.nn.functional as F

In [24]:
num_epochs = 10

for epoch in range(num_epochs):  
    running_loss, running_items, running_right = 0.0, 0.0, 0.0
    for i, data in enumerate(train_loader):
        inputs, labels = data[0], data[1]

        # обнуляем градиент
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # выводим статистику о процессе обучения
        running_loss += loss.item()
        running_items += len(labels)
#        running_right += (labels == torch.max(outputs, 1)[1]).sum()
        
        # выводим статистику о процессе обучения
        if i % 50 == 0:    # печатаем каждые 50 mini-batches
            net.eval()
            
            print(f'Epoch [{epoch + 1}/{num_epochs}]. ' \
                  f'Step [{i + 1}/{len(train_loader)}]. ' \
                  f'Loss: {running_loss / running_items:.3f}. ' \
                  f'Acc: {running_right / running_items:.3f}', end='. ')
            running_loss, running_items, running_right = 0.0, 0.0, 0.0

            test_running_right, test_running_total = 0.0, 0.0
            for i, data in enumerate(test_loader):
                running_loss += loss.item()
                test_outputs = net(data[0])
                test_running_total += len(data[1])
                test_running_right += (data[1] == torch.max(test_outputs, 1)[1]).sum()
            
            print(f'Test acc: {test_running_right / test_running_total:.3f}')
            net.train()
        
print('Training is finished!')

Epoch [1/10]. Step [1/121]. Loss: 0.035. Acc: 0.000. Test acc: 0.000
Epoch [1/10]. Step [51/121]. Loss: 0.394. Acc: 0.000. Test acc: 0.000
Epoch [1/10]. Step [101/121]. Loss: 0.158. Acc: 0.000. Test acc: 0.000
Epoch [2/10]. Step [1/121]. Loss: 0.013. Acc: 0.000. Test acc: 0.000
Epoch [2/10]. Step [51/121]. Loss: 0.143. Acc: 0.000. Test acc: 0.000
Epoch [2/10]. Step [101/121]. Loss: 0.122. Acc: 0.000. Test acc: 0.000
Epoch [3/10]. Step [1/121]. Loss: 0.010. Acc: 0.000. Test acc: 0.000
Epoch [3/10]. Step [51/121]. Loss: 0.110. Acc: 0.000. Test acc: 0.000
Epoch [3/10]. Step [101/121]. Loss: 0.121. Acc: 0.000. Test acc: 0.000
Epoch [4/10]. Step [1/121]. Loss: 0.010. Acc: 0.000. Test acc: 0.000
Epoch [4/10]. Step [51/121]. Loss: 0.117. Acc: 0.000. Test acc: 0.000
Epoch [4/10]. Step [101/121]. Loss: 0.134. Acc: 0.000. Test acc: 0.000
Epoch [5/10]. Step [1/121]. Loss: 0.010. Acc: 0.000. Test acc: 0.000
Epoch [5/10]. Step [51/121]. Loss: 0.115. Acc: 0.000. Test acc: 0.000
Epoch [5/10]. Step [1

*Adam*

In [25]:
net = FeedForward(length, batch_size*8)
optimizer = torch.optim.Adam(net.parameters(), lr=0.1)
criterion = torch.nn.MSELoss()

In [26]:
num_epochs = 10

for epoch in range(num_epochs):  
    running_loss, running_items, running_right = 0.0, 0.0, 0.0
    for i, data in enumerate(train_loader):
        inputs, labels = data[0], data[1]

        # обнуляем градиент
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # выводим статистику о процессе обучения
        running_loss += loss.item()
        running_items += len(labels)
#        running_right += (labels == torch.max(outputs, 1)[1]).sum()
        
        # выводим статистику о процессе обучения
        if i % 50 == 0:    # печатаем каждые 50 mini-batches
            net.eval()
            
            print(f'Epoch [{epoch + 1}/{num_epochs}]. ' \
                  f'Step [{i + 1}/{len(train_loader)}]. ' \
                  f'Loss: {running_loss / running_items:.3f}. ' \
                  f'Acc: {running_right / running_items:.3f}', end='. ')
            running_loss, running_items, running_right = 0.0, 0.0, 0.0

            test_running_right, test_running_total = 0.0, 0.0
            for i, data in enumerate(test_loader):
                running_loss += loss.item()
                test_outputs = net(data[0])
                test_running_total += len(data[1])
                test_running_right += (data[1] == torch.max(test_outputs, 1)[1]).sum()
            
            print(f'Test acc: {test_running_right / test_running_total:.3f}')
            net.train()
        
print('Training is finished!')

Epoch [1/10]. Step [1/121]. Loss: 0.042. Acc: 0.000. Test acc: 0.000
Epoch [1/10]. Step [51/121]. Loss: 0.443. Acc: 0.000. Test acc: 0.000
Epoch [1/10]. Step [101/121]. Loss: 0.119. Acc: 0.000. Test acc: 0.000
Epoch [2/10]. Step [1/121]. Loss: 0.011. Acc: 0.000. Test acc: 0.000
Epoch [2/10]. Step [51/121]. Loss: 0.120. Acc: 0.000. Test acc: 0.000
Epoch [2/10]. Step [101/121]. Loss: 0.129. Acc: 0.000. Test acc: 0.000
Epoch [3/10]. Step [1/121]. Loss: 0.010. Acc: 0.000. Test acc: 0.000
Epoch [3/10]. Step [51/121]. Loss: 0.116. Acc: 0.000. Test acc: 0.000
Epoch [3/10]. Step [101/121]. Loss: 0.138. Acc: 0.000. Test acc: 0.000
Epoch [4/10]. Step [1/121]. Loss: 0.009. Acc: 0.000. Test acc: 0.000
Epoch [4/10]. Step [51/121]. Loss: 0.102. Acc: 0.000. Test acc: 0.000
Epoch [4/10]. Step [101/121]. Loss: 0.129. Acc: 0.000. Test acc: 0.000
Epoch [5/10]. Step [1/121]. Loss: 0.012. Acc: 0.000. Test acc: 0.000
Epoch [5/10]. Step [51/121]. Loss: 0.131. Acc: 0.000. Test acc: 0.000
Epoch [5/10]. Step [1

*RMSProp*

In [27]:
net = FeedForward(length, batch_size*8)
optimizer = torch.optim.RMSprop(net.parameters(), lr=0.1, alpha=0.99)
criterion = torch.nn.MSELoss()

In [28]:
num_epochs = 10

for epoch in range(num_epochs):  
    running_loss, running_items, running_right = 0.0, 0.0, 0.0
    for i, data in enumerate(train_loader):
        inputs, labels = data[0], data[1]

        # обнуляем градиент
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # выводим статистику о процессе обучения
        running_loss += loss.item()
        running_items += len(labels)
#        running_right += (labels == torch.max(outputs, 1)[1]).sum()
        
        # выводим статистику о процессе обучения
        if i % 50 == 0:    # печатаем каждые 50 mini-batches
            net.eval()
            
            print(f'Epoch [{epoch + 1}/{num_epochs}]. ' \
                  f'Step [{i + 1}/{len(train_loader)}]. ' \
                  f'Loss: {running_loss / running_items:.3f}. ' \
                  f'Acc: {running_right / running_items:.3f}', end='. ')
            running_loss, running_items, running_right = 0.0, 0.0, 0.0

            test_running_right, test_running_total = 0.0, 0.0
            for i, data in enumerate(test_loader):
                running_loss += loss.item()
                test_outputs = net(data[0])
                test_running_total += len(data[1])
                test_running_right += (data[1] == torch.max(test_outputs, 1)[1]).sum()
            
            print(f'Test acc: {test_running_right / test_running_total:.3f}')
            net.train()
        
print('Training is finished!')

Epoch [1/10]. Step [1/121]. Loss: 0.041. Acc: 0.000. Test acc: 0.000
Epoch [1/10]. Step [51/121]. Loss: 0.555. Acc: 0.000. Test acc: 0.000
Epoch [1/10]. Step [101/121]. Loss: 0.491. Acc: 0.000. Test acc: 0.000
Epoch [2/10]. Step [1/121]. Loss: 0.048. Acc: 0.000. Test acc: 0.000
Epoch [2/10]. Step [51/121]. Loss: 0.537. Acc: 0.000. Test acc: 0.000
Epoch [2/10]. Step [101/121]. Loss: 0.412. Acc: 0.000. Test acc: 0.000
Epoch [3/10]. Step [1/121]. Loss: 0.044. Acc: 0.000. Test acc: 0.000
Epoch [3/10]. Step [51/121]. Loss: 0.496. Acc: 0.000. Test acc: 0.000
Epoch [3/10]. Step [101/121]. Loss: 0.548. Acc: 0.000. Test acc: 0.000
Epoch [4/10]. Step [1/121]. Loss: 0.039. Acc: 0.000. Test acc: 0.000
Epoch [4/10]. Step [51/121]. Loss: 0.446. Acc: 0.000. Test acc: 0.000
Epoch [4/10]. Step [101/121]. Loss: 0.491. Acc: 0.000. Test acc: 0.000
Epoch [5/10]. Step [1/121]. Loss: 0.043. Acc: 0.000. Test acc: 0.000
Epoch [5/10]. Step [51/121]. Loss: 0.491. Acc: 0.000. Test acc: 0.000
Epoch [5/10]. Step [1

ВЫВОДЫ: к сожалению так до конца и не разобрался, как применять pytorch на табличных данных

Буду очень признателен за предоставление доп информации, чтобы разобраться с этим (примеры ноутбуков других студентов как вариант).
Заранее спасибо!